In [1]:
import os
import b3d
import jax.numpy as jnp
import numpy as np
import h5py
import matplotlib.pyplot as plt



In [9]:
# paths for reading physion metadata
physion_assets_path = os.path.join(
    b3d.get_root_path(),
    "assets/physion/",)

hdf5_file_path = os.path.join(physion_assets_path,
    "hdf5s/pilot_dominoes_0mid_d3chairs_o1plants_tdwroom_0012.hdf5",
)

video_file_path = os.path.join(physion_assets_path,
    "videos/pilot_dominoes_0mid_d3chairs_o1plants_tdwroom_0012_img.mp4",
)

In [32]:
def compute_angular_velocity(q1, q2, dt):
    return (2 / dt) * np.array([
        q1[0]*q2[1] - q1[1]*q2[0] - q1[2]*q2[3] + q1[3]*q2[2],
        q1[0]*q2[2] + q1[1]*q2[3] - q1[2]*q2[0] - q1[3]*q2[1],
        q1[0]*q2[3] - q1[1]*q2[2] + q1[2]*q2[1] - q1[3]*q2[0]])

# Compute angular velocity
angular_velocity = compute_angular_velocity(q1, q2, dt)
print("Angular Velocity:", angular_velocity)

Angular Velocity: [-7.6536504  0.         0.       ]


In [36]:
fps = b3d.io.utils.load_video_info(video_file_path).fps
delta_t = 1/fps
obj_idx = 2

with h5py.File(hdf5_file_path, "r") as f:
    for key in f['frames'].keys():
        ang_vel = jnp.array(f['frames'][key]['objects']['angular_velocities'])
        quaternion_now = jnp.array(f['frames'][key]['objects']['rotations_cam0'])
        # print(ang_vel.shape, quaternion_now.shape)
        if key == '0000':
            continue
        frame_idx = int(key)
        quaternion_last = jnp.array(f['frames'][str(frame_idx-1).zfill(4)]['objects']['rotations_cam0'])
        angular_velocity = compute_angular_velocity(quaternion_last[obj_idx], quaternion_now[obj_idx], delta_t)
        angular_velocity = np.dot(angular_velocity, np.array([[0,0,1],[0,-1,0],[-1,0,0]]))
        # angular_velocity[[2,0]] = -angular_velocity[[0,2]]
        print("Angular Velocity computed:", angular_velocity)
        print("Angular Velocity recorded:", ang_vel[obj_idx], '\n')

Angular Velocity computed: [3.63158688e-06 0.00000000e+00 2.11260567e-06]
Angular Velocity recorded: [2.6770783e-07 5.3042413e-08 1.5044325e-06] 

Angular Velocity computed: [2.95259997e-06 1.49011612e-06 1.30721071e-06]
Angular Velocity recorded: [-3.7298349e-08  2.2153611e-08  3.3657727e-07] 

Angular Velocity computed: [-2.75192906e-06  0.00000000e+00 -1.08934341e-06]
Angular Velocity recorded: [-6.3725224e-07  5.0274025e-09 -9.5760515e-07] 

Angular Velocity computed: [ 1.19738820e-06  0.00000000e+00 -6.45829687e-07]
Angular Velocity recorded: [ 1.0213432e-06 -3.2297285e-09 -3.2911703e-07] 

Angular Velocity computed: [ 1.04804485e-06  0.00000000e+00 -2.79053040e-07]
Angular Velocity recorded: [ 1.0518171e-06 -2.2217783e-12 -2.6270243e-07] 

Angular Velocity computed: [ 5.96975269e-07  0.00000000e+00 -4.59317306e-07]
Angular Velocity recorded: [ 3.391466e-07  5.340818e-09 -7.445982e-07] 

Angular Velocity computed: [-5.61142599e-07  0.00000000e+00 -2.15265317e-07]
Angular Velocity 